In [1]:
import os
import sys
import glob
import json
import numpy as np
from datetime import datetime
from sklearn.metrics import f1_score

from aeronet.dataset import Predictor
from aeronet.dataset import BandCollection, parse_directory
from PIL import Image
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from segmentation_models import Unet
from tensorflow.keras.layers import Input
from segmentation_models.backbones import get_preprocessing
from segmentation_models.losses import bce_jaccard_loss
from segmentation_models.metrics import get_jaccard_score

sys.path.append("..")
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
PREDICTION_ROOT = "./prediction/"

Using TensorFlow backend.
/root/anaconda3/envs/base/lib/python3.7/site-packages/classification_models/resnext/__init__.py:4: UserWarning: Current ResNext models are deprecated, use keras.applications ResNeXt models
  warnings.warn('Current ResNext models are deprecated, '


In [2]:
from tta_wrapper import tta_segmentation

In [4]:
from src.utils.inference_util import InferenceModel
from src.utils.f1_util import get_f1_score, convert_tif_to_geojson
from src.visualization.visualization_util import show_images

In [5]:
gpu_options = tf.compat.v1.GPUOptions(allow_growth=True)
session = tf.compat.v1.InteractiveSession(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

In [9]:
def preproc_irv2(x):
    x[:,:,:, :][x[:,:,:, :]<0] = 0
    x[:,:,:, :][x[:,:,:, :]>1] = 1
    
    return x

In [11]:
def dice_coef(y_true, y_pred, smooth=1):
    """
    Dice = (2*|X & Y|)/ (|X|+ |Y|)
         =  2*sum(|A*B|)/(sum(A^2)+sum(B^2))
    ref: https://arxiv.org/pdf/1606.04797v1.pdf
    """
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    return (2. * intersection + smooth) / (K.sum(K.square(y_true),-1) + K.sum(K.square(y_pred),-1) + smooth)

def dice_coef_loss(y_true, y_pred):
    if len(y_true)==2:
        y_tr = tf.expand_dims(y_true[0], axis=-1)
    else:
        y_tr = y_true
    return 1-dice_coef(y_tr, y_pred)

In [12]:
dependencies = {
    "dice_coef_loss": dice_coef_loss,
    "score":get_jaccard_score(per_image=False)
}

## Models and Data settings

In [13]:
inceptionresnetv2_filtersX2 = load_model("/root/les/models/satellite/224/inceptionresnetv2_306_0.850.h5", custom_objects=dependencies)
inceptionresnetv2_filtersX2_tta = tta_segmentation(inceptionresnetv2_filtersX2, h_flip=True, v_flip=True, rotation=(90, 180, 270), merge='mean')

In [15]:
DATA_ROOT_DIR = "/root/les/forest_segmentation/TestData"
DATA_DIRS = [os.path.join(DATA_ROOT_DIR, f, "AnalyticalProducts") for f in os.listdir(DATA_ROOT_DIR) if f!="FoldersStructureTemplate" and f!="train_samples"]
TEST_DATA_DIRS_AND_SHAPES = sorted([os.path.join(f, os.listdir(f)[0]) for f in DATA_DIRS ])

In [16]:
TEST_DATA_DIRS_AND_SHAPES

['/root/les/forest_segmentation/TestData/1/AnalyticalProducts/2020-09-13_07:22:57_S2A_MSIL1C_20190605T072621_N0207_R049_T40VDJ_20190605T094322.SAFE_QuarryPostProcessorTask_5f796aa6-9811-4546-a3f8-d144c3938124',
 '/root/les/forest_segmentation/TestData/2/AnalyticalProducts/2020-09-13_07:23:18_S2B_MSIL1C_20190511T072629_N0207_R049_T40VDJ_20190511T093112.SAFE_QuarryPostProcessorTask_5d46771b-4980-46e0-b219-bc74e7b3aaf5']

In [17]:
CHANNLES = ["B04", "B03", "B02"]
LABELS = ['100']

SAMPLE_SIZE = (224, 224)

## Test inference

In [19]:
predictions_and_gts = []
model = {"name": "inceptionresnetv2_filtersX2_tta", "model": inceptionresnetv2_filtersX2_tta, "preproc": preproc_irv2, "postproc": None}

for test_data in TEST_DATA_DIRS_AND_SHAPES:
    # # read test data
    x_test = BandCollection(parse_directory(test_data, CHANNLES))
    y_test = BandCollection(parse_directory(test_data, LABELS))
    
    
    # create infernce model and predictor object
    inference_model = InferenceModel(model["model"], model["preproc"], model["postproc"])

    # `predictor` object process `band collection` with `inference model` 
    # usind sliding window approach 
    # (nesessery for big raster files which are not fit to RAM)
    predictor = Predictor(
        input_channels=CHANNLES,
        output_labels=LABELS,
        processing_fn=inference_model, 
        sample_size=(1024, 1024),
        bound=64, 
        dtype=np.float32
    )

    # run band collection processing
    output_directory = os.path.join(PREDICTION_ROOT, os.path.basename(test_data), model["name"])
    pred = predictor.process(bc=x_test, output_directory=output_directory)
    convert_tif_to_geojson(os.path.join(output_directory, "100.tif"), os.path.join(output_directory, "100.geojson"))


  0%|          | 0/121 [00:00<?, ?it/s]

(1, 1152, 1152, 3)


  1%|          | 1/121 [00:15<31:31, 15.76s/it]

(1, 1152, 1152, 3)


  2%|▏         | 2/121 [00:16<22:30, 11.35s/it]

...

(1, 1152, 1152, 3)


 99%|█████████▉| 120/121 [02:19<00:01,  1.05s/it]

(1, 1152, 1152, 3)


100%|██████████| 121/121 [02:20<00:00,  1.16s/it]


KeyboardInterrupt: 